In [169]:
import numpy as np

In [170]:
N = 5 # teachers
M = 2 # subjects per teacher
K = 2 # groups
W = 2 # weeks
T = 8 # hours per day
D = 6 # 6 days per week

# indices: [g, w, d, t, p, s]
sizes = [K, W, D, T, N, M]

In [171]:
# function for calculating the flattened index
def calculateindex(offsets, sizes):
    total_index = 0
    for offset, size in zip(offsets, sizes):
        total_index *= size
        total_index += offset
    return total_index

In [172]:
# penaltiy coefficients

A_curr = 10000 # curriculum requirements
A_off = 5000 # teachers' days off

penalty_teacher_simultaneous = 5000 # one teacher at a time
penalty_student_simultaneous = 5000 # one lecture for a group at a time

penalty_student_overload_1 = 69 # no more than X lectures per day
penalty_student_overload_2 = 1 # no more than X lectures per day

In [173]:
# общее число пар по предмету в две недели 4 
XX1 = np.zeros([K*W*D*T*N*M, K*W*D*T*N*M])

for g in range(K):
    for p in range(N):
        for s in range(M):
            for w in range(W):
                for d in range(D):
                    for t in range(T):
                        for w1 in range(W):
                            for d1 in range(D):
                                for t1 in range(T):
                                    if ((w==w1) & (d==d1) & (t==t1)):
                                        XX1[calculateindex([g, w, d, t, p, s], sizes), 
                                            calculateindex([g, w, d, t, p, s], sizes)] = -7*A_curr
                                    else:
                                        XX1[calculateindex([g, w, d, t, p, s], sizes), 
                                            calculateindex([g, w1, d1, t1, p, s], sizes)] = A_curr

In [174]:
# профессор 1 не ведет по ср, ...
# массив нерабочих дней преподавателей
XX2 = np.zeros([K*W*D*T*N*M, K*W*D*T*N*M])
non_working_days = [2, 0, 5, 1, 1]

qubo = np.zeros([K*W*D*T*N*M, K*W*D*T*N*M])
for g in range(K):
    for w in range(W):
        #d = 2
        for t in range(T):
            #p = 0
            for s in range(M):
                for prof, day in enumerate(non_working_days):
                    XX2[calculateindex([g, w, day, t, prof, s], sizes), 
                                            calculateindex([g, w, day, t, prof, s], sizes)] = A_off

In [175]:
# не учит одновременно 2 группы или не ведет 2 занятия

alpha = 2 # счетчик неравенства
XXtd = np.zeros([K*W*D*T*N*M, K*W*D*T*N*M])
XYtd = np.zeros([K*W*D*T*N*M,W*D*T*N*alpha])
YYtd = np.zeros([W*D*T*N*alpha, W*D*T*N*alpha])

for w in range(W):
    for d in range(D):
        for t in range(T):
            for p in range(N):
                for g in range(K):
                    for s in range(M):
                        for g1 in range(K):
                            for s1 in range(M):
                                XXtd[calculateindex([g, w, d, t, p, s], sizes), 
                                            calculateindex([g1, w, d, t, p, s1], sizes)] = penalty_teacher_simultaneous
                                
for w in range(W):
    for d in range(D):
        for t in range(T):
            for p in range(N):
                for g in range(K):
                    for s in range(M):
                        for alpha in range(2):
                                        XYtd[calculateindex([g, w, d, t, p, s], sizes), 
                                            calculateindex([w, d, t, p, alpha], [W,D,T,N,2])] = -penalty_teacher_simultaneous*alpha
                                        
                                        
for w in range(W):
    for d in range(D):
        for t in range(T):
            for p in range(N):
                for alpha in range(2):
                    for alpha1 in range(2):
                        if (alpha == alpha1):
                            YYtd[calculateindex([w, d, t, p, alpha], [W,D,T,N,2]), 
                                    calculateindex([w, d, t, p, alpha], [W,D,T,N,2])] = -penalty_teacher_simultaneous+alpha**2*penalty_teacher_simultaneous
                        else:
                            YYtd[calculateindex([w, d, t, p, alpha], [W,D,T,N,2]), 
                                    calculateindex([w, d, t, p, alpha1], [W,D,T,N,2])] = penalty_teacher_simultaneous+alpha*alpha1*penalty_teacher_simultaneous

In [176]:
# студент не может быть на 2х парах

alphas = 2 # счетчик неравенства
XXsd = np.zeros([K*W*D*T*N*M, K*W*D*T*N*M])
XYsd = np.zeros([K*W*D*T*N*M,K*W*D*T*alphas])
YYsd = np.zeros([K*W*D*T*alphas, K*W*D*T*alphas])

for g in range(K):
    for w in range(W):
        for d in range(D):
            for t in range(T):
                for p in range(N):
                    for s in range(M):
                        for p1 in range(N):
                            for s1 in range(M):
                                XXsd[calculateindex([g, w, d, t, p, s], sizes), 
                                            calculateindex([g, w, d, t, p1, s1], sizes)] = penalty_student_simultaneous
                                

for g in range(K):                                
    for w in range(W):
        for d in range(D):
            for t in range(T):
                for p in range(N):
                    for s in range(M):
                        for alpha in range(alphas):
                            XYsd[calculateindex([g, w, d, t, p, s], sizes), 
                                            calculateindex([g,w, d, t, alpha], [K,W,D,T,alphas])] = -penalty_student_simultaneous*alpha
                                        
                                        
for g in range(K):
    for w in range(W):
        for d in range(D):
            for t in range(T):
                for alpha in range(alphas):
                    for alpha1 in range(alphas):
                        if (alpha == alpha1):
                            YYsd[calculateindex([g,w, d, t,  alpha], [K,W,D,T,alphas]), 
                                    calculateindex([g,w, d, t,  alpha], [K,W,D,T,alphas])] = -penalty_student_simultaneous+alpha**2*penalty_student_simultaneous
                        else:
                            YYsd[calculateindex([g,w, d, t,  alpha], [K,W,D,T,alphas]), 
                                    calculateindex([g,w, d, t,  alpha1], [K,W,D,T,alphas])] = penalty_student_simultaneous+alpha*alpha1*penalty_student_simultaneous

In [177]:
al = 7 # счетчик неравенства
XXso = np.zeros([K*W*D*T*N*M, K*W*D*T*N*M])
XYso = np.zeros([K*W*D*T*N*M, K*W*D*al])
YYso = np.zeros([K*W*D*al, K*W*D*al])

for g in range(K):
    for w in range(W):
        for d in range(D):
            for t in range(T):
                for p in range(N):
                    for s in range(M):
                        for t1 in range(T):
                            for p1 in range(N):
                                for s1 in range(M):
                                    XXso[calculateindex([g, w, d, t, p, s], sizes), calculateindex([g, w, d, t1, p1, s1], sizes)] += penalty_student_overload_2

for g in range(K):                                
    for w in range(W):
        for d in range(D):
            for t in range(T):
                for p in range(N):
                    for s in range(M):
                        for alpha in range(al):
                            XYso[calculateindex([g, w, d, t, p, s], sizes), calculateindex([g,w, d, alpha], [K,W,D,al])] += -penalty_student_overload_2*alpha

for g in range(K):
    for w in range(W):
        for d in range(D):
            for alpha in range(al):
                for alpha1 in range(al):
                    if (alpha == alpha1):
                        YYso[calculateindex([g, w, d, alpha], [K,W,D,al]), calculateindex([g,w, d,  alpha], [K,W,D,al])] += -penalty_student_overload_1+alpha**2*penalty_student_overload_2
                    else:
                        YYso[calculateindex([g, w, d, alpha], [K,W,D,al]), calculateindex([g,w, d,  alpha1], [K,W,D,al])] += penalty_student_overload_1+alpha*alpha1*penalty_student_overload_2

In [179]:
# comprise the whole matrix
qubo = np.block([[XX1+XX2+XXtd+XXsd+XXso, XYtd,   XYsd,   XYso],
    [XYtd.T, YYtd,   np.zeros((np.shape(YYtd)[1],np.shape(XYsd)[1])),    np.zeros((np.shape(YYtd)[1],np.shape(XYso)[1]))],
    [XYsd.T, np.zeros((np.shape(YYsd)[1],np.shape(XYtd)[1])), YYsd ,   np.zeros((np.shape(YYsd)[1],np.shape(XYso)[1]))],
    [XYso.T, np.zeros((np.shape(YYso)[1],np.shape(XYtd)[1])),   np.zeros((np.shape(YYso)[1],np.shape(XYsd)[1])),   YYso],             
    ])

In [180]:
np.save('schedule_QUBO.npy', qubo)